In [1]:
!pip install chromedriver_autoinstaller

In [2]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
# python 버전 확인
!python --version

Python 3.9.7


In [4]:
# 판다스 버전 확인
pd.__version__

'1.3.4'

In [5]:
# !pip install selenium

In [6]:
# 검색어, 검색 기간 정의
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

1.크롤링할 키워드는 무엇입니까?: 데이터


In [7]:
query_txt

'데이터'

# 1. 크롤링할 블로그 url 수집하기

In [45]:
pwd

'C:\\Users\\eunwon\\Desktop\\크롤링'

In [46]:
# !pip install selenium

In [1]:
# Step 1. 크롬 웹브라우저 실행
# pip install chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 사이트 주소는 네이버
driver.get('https://www.work.go.kr/')
time.sleep(2)  # 2초간 정지

NameError: name 'chromedriver_autoinstaller' is not defined

In [48]:
# Step 2. 워크넷검색창에 "검색어" 검색
element = driver.find_element_by_id("topQuery")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드

In [49]:
#검색어 검색
search = driver.find_element_by_css_selector(".btn-search")
search.click()
time.sleep(1)

In [50]:
# '상세검색' 클릭
driver.find_element_by_link_text("상세검색").click( )
time.sleep(1)

In [51]:
# 스크롤 다운
# driver.execute_script("window.scrollTo(0, 500)")
# time.sleep(2)

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 1
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [52]:
# '제목' 클릭
driver.find_element_by_id("srckeywordWantedTitle").click( )
time.sleep(1)

In [53]:
# '고용형태' 클릭
driver.find_element_by_id("employGbnParam10").click( )
time.sleep(1)

In [54]:
# '검색' 클릭
driver.find_element_by_css_selector(".right").click( )
time.sleep(1)

In [55]:
# '간편보기' 클릭
driver.find_element_by_id("viewSPL").click( )
time.sleep(1)

In [57]:
#공모건수 찾기
number=driver.find_element_by_css_selector(".futura")
count=int(number.text)
count

In [58]:
# 크롤링

url_list = []
title_list = []
cp_name_list=[]
cp_info_list=[]
cp_term_list=[]
date_list=[]

for j in tqdm_notebook(range(0,count//10+1)):    
    recruit = ".cp-info-in>a"
    recruit_raw = driver.find_elements_by_css_selector(recruit)
    cp_name_raw = driver.find_elements_by_css_selector(".cp_name")
    cp_info_raw=driver.find_elements_by_css_selector(".cp-info")
                                   
    for recruit in recruit_raw:
        title = recruit.text
        title_list.append(title)

        url = recruit.get_attribute('href')
        url_list.append(url)
        time.sleep(1)

     
    for cpn in cp_name_raw:
 
        cp_name = cpn.text
        cp_name_list.append(cp_name)
        time.sleep(1)
        


    for i in range(0,30):
  #4.근무내용
        if i%3==0 :
            cp_info =cp_info_raw[i].text
            cp_info_list.append(cp_info)
            time.sleep(1)

    #5근무조건
        elif i%3==1 : 
            cp_term = cp_info_raw[i].text
            cp_term_list.append(cp_term)
            time.sleep(1)

    #6.등록마감
        else:
            date = cp_info_raw[i].text
            date_list.append(date)



    element=driver.find_element_by_id("currentPageNo")
    element.click
    element.clear()
    time.sleep(1)  
    element.send_keys(j+1)  
    go=driver.find_element_by_css_selector(".btn_direct")
    go.click()




  0%|          | 0/14 [00:00<?, ?it/s]

In [59]:
#갯수확인
print(len(title_list))
print(len(url_list))
print(len(cp_name_list))
print(len(cp_info_list))
print(len(cp_term_list))
print(len(date_list))


140
140
139
140
140
140


In [60]:
#데이터프레임만들기
import pandas as pd
df = pd.DataFrame({"title":title_list,"cp_name":cp_name_list,"cp_info":cp_info_list,"cp_term":cp_term_list,"date":date_list,"url":url_list})
df

ValueError: All arrays must be of the same length

In [ ]:
# 저장하기
df.to_excel("worknet_content.xlsx", encoding='utf-8-sig')      

In [100]:
cp_name_list


['(주)써치포리더스',
 '(주)웨어밸리데이타',
 '주식회사 베니트글로벌',
 '(주)은광전자',
 '(주)디에이치테크',
 '㈜월드피에이디',
 '(주)모핑아이',
 '㈜테크다스',
 '안진회계법인',
 '(주)인코리아 프로페셔널',
 '(주)써치포리더스',
 '(주)웨어밸리데이타',
 '주식회사 베니트글로벌',
 '(주)은광전자',
 '(주)디에이치테크',
 '㈜월드피에이디',
 '(주)모핑아이',
 '㈜테크다스',
 '안진회계법인',
 '(주)인코리아 프로페셔널',
 '(주)디에스랩글로벌',
 '주식회사에이아이앤잡',
 '보성파워텍(주)',
 '(주)엑셀리언트',
 '(주)이지서티',
 '(주)이지서티',
 '(주)펫온',
 '㈜호디',
 '쿼드라컨트롤서비스(주)',
 '(주)비포플레이',
 '(주) 메드고',
 '데이터메쉬(주)',
 '(주)어큐진',
 '㈜크린토피아',
 '신라정보기술(주)',
 '(주)신라아이티',
 '(주)솔루파인',
 '주식회사엠아이티능력개발원',
 '(주)스카우트파트너스',
 '(주)아이비엘글로벌',
 '㈜잡스랩',
 '(주)제이씨그룹',
 '(주)타이로스코프',
 '디케이테크인',
 '씨앤토트플러스(주)',
 '㈜연합인포맥스',
 '휴머레인컨설팅',
 '휴머레인컨설팅',
 '주식회사 라피플코리아리미티드',
 '(주)민앤지',
 '나이스씨엠에스㈜',
 '(주)아이디케이',
 '비즈그리다(주)',
 '(주)지오비전',
 '주식회사 여기어때컴퍼니',
 '(주)라온로드',
 '히어로웍스',
 '㈜해커스어학원',
 '이프린팅 주식회사',
 '(주)웹스시스템코리아',
 'NKD',
 '(주)지오비전',
 '㈜로드텍',
 '비즈빌커뮤니케이션',
 '주식회사 투비유니콘',
 '한솔엠티에스(주)',
 '(주)디어유',
 '테크빌교육(주)',
 '(주)퍼닌',
 '(주)에버원소프트',
 '주)어빌리티시스템즈',
 '㈜더볼',
 '(주)티지360테크놀로지스',
 '주식회사 에픽타이거',
 '(주)올위메이크이즈',
 '(주)브릿지미디어그룹',
